In [1]:
from xtb_density import CDCalculator

In [2]:
claculator = CDCalculator()

In [3]:
from rdkit import Chem

In [4]:
mol = Chem.MolFromMolFile('./0.sdf')

In [5]:
ecloud = claculator.calculate(mol)

In [6]:
ecloud['meta']

{'org': array([-6.69103144, -4.89703166, -3.68583172]),
 'xvec': [0.956569, 0.0, 0.0],
 'yvec': [0.0, 0.952135, 0.0],
 'zvec': [0.0, 0.0, 0.956453],
 'nx': 26,
 'ny': 19,
 'nz': 15,
 'len': array([13.16105835,  9.57311542,  7.59199751])}

In [9]:
ecloud['density'].shape

(26, 19, 15)

In [10]:
import numpy as np

In [11]:
np.meshgrid([3,4])

[array([3, 4])]

In [62]:
import numpy as np

x_len, y_len, z_len = ecloud['meta']['org']

# 三个维度上cell的数量分别为 x_cell、y_cell、z_cell
x_cell, y_cell, z_cell = ecloud['meta']['nx'], ecloud['meta']['ny'], ecloud['meta']['nz']

# 使用 np.meshgrid 创建 X、Y、Z 三个网格数组
X, Y, Z = np.meshgrid(np.arange(float(x_cell)), np.arange(float(y_cell)), np.arange(float(z_cell)))

# 将每个网格点乘以网格密度，也就是每个 cell 的大小
X *= x_len / x_cell
Y *= y_len / float(y_cell)
Z *= z_len / float(z_cell)

# 将三个网格数组沿着最后一个轴合并成一个数组
grid_coords = np.stack([X, Y, Z], axis=-1)

In [70]:
ecloud['density'].shape

(26, 19, 15)

In [71]:
grid_coords.shape

(19, 26, 15, 3)

In [80]:
grid_coords.reshape(x_cell*y_cell*z_cell, -1)[10]

array([-0.        , -0.        , -2.45722115])

In [81]:
ecloud['density'].reshape(x_cell*y_cell*z_cell, -1)[10]

array([3.77821844e-11])

In [48]:
import torch

In [49]:
grid_coords = torch.tensor(grid_coords)

In [55]:
grid_coords.shape

torch.Size([19, 26, 15, 3])

In [9]:
import numpy as np
lig_ocup = np.load('./lig_occup.npy')
pkt_ocup = np.load('./pkt_occup.npy')

In [11]:
BOHR = 1.8897259886

In [13]:
ecloud['meta']

{'org': array([-6.69103144, -4.89703166, -3.68583172]),
 'xvec': [0.956569, 0.0, 0.0],
 'yvec': [0.0, 0.952135, 0.0],
 'zvec': [0.0, 0.0, 0.956453],
 'nx': 26,
 'ny': 19,
 'nz': 15,
 'len': array([13.16105835,  9.57311542,  7.59199751])}

In [17]:
from cubtools import *
cube_parser = CubeFile('392d22118eef43b8919a03872ca44365/density.cub')

In [119]:
def _putline(*args):
    """
    Generate a line to be written to a cube file where
    the first field is an int and the remaining fields are floats.

    params:
        *args: first arg is formatted as int and remaining as floats

    returns: formatted string to be written to file with trailing newline
    """
    s = "{0:^ 8d}".format(args[0])
    s += "".join("{0:< 12.6f}".format(arg) for arg in args[1:])
    return s + "\n"

def write_cube(fname, meta, data):
    with open(fname, "w") as cube:
        # first two lines are comments
        cube.write(" Cubefile created by cubetools.py\n  source: none\n")
        natm = len(meta['atoms'])
        nx, ny, nz = data.shape
        cube.write(_putline(natm, *meta['org'])) # 3rd line #atoms and origin
        cube.write(_putline(nx, *meta['xvec']))
        cube.write(_putline(ny, *meta['yvec']))
        cube.write(_putline(nz, *meta['zvec']))
        for atom_mass, atom_pos in meta['atoms']:
            cube.write(_putline(atom_mass, *atom_pos)) #skip the newline
        for i in range(nx):
            for j in range(ny):
                for k in range(nz):
                    if (i or j or k) and k%6==0:
                        cube.write("\n")
                    cube.write(" {0: .5E}".format(data[i,j,k]))

def write_dualcube(pkt_mol, lig_mol, pkt_data, lig_data):

    lig_conf = lig_mol.GetConformer(0).GetPositions() *BOHR 
    lig_atoms = [(int(atom.GetMass()), lig_conf[atom.GetIdx()]) for atom in lig_mol.GetAtoms()]
    
    pkt_conf = pkt_mol.GetConformer(0).GetPositions() *BOHR
    pkt_atoms = [(int(atom.GetMass()), pkt_conf[atom.GetIdx()]) for atom in pkt_mol.GetAtoms()]
    pkt_center = pkt_conf.mean(axis=0)

    pkt_meta = {'org':np.array([-12,-12,-12])*BOHR + pkt_center, \
        'xvec':[1.0*BOHR, 0.0, 0.0],'yvec':[0.0, 1.0*BOHR, 0.0],'zvec':[0.0, 0.0, 1.0*BOHR],\
            'nx':np.array(24),'ny':np.array(24),'nz':np.array(24),\
                'atoms':pkt_atoms}

    lig_meta = {'org':np.array([-12,-12,-12])*BOHR + pkt_center, \
        'xvec':[1.0*BOHR, 0.0, 0.0],'yvec':[0.0, 1.0*BOHR, 0.0],'zvec':[0.0, 0.0, 1.0*BOHR],\
            'nx':np.array(24),'ny':np.array(24),'nz':np.array(24),\
                'atoms':lig_atoms}
    
    write_cube('./pkt.cub',pkt_meta,pkt_data)
    write_cube('./lig.cub',lig_meta,lig_data)

In [120]:
pkt_data = np.load('./pkt_occup.npy')
lig_data = np.load('./lig_occup.npy')
lig_mol = Chem.MolFromMolFile('./lig.sdf')
pkt_mol = Chem.MolFromMolFile('./pkt.sdf')
write_dualcube(pkt_mol, lig_mol, pkt_data, lig_data)

In [121]:
Chem.MolToPDBFile(pkt_mol, './pkt.pdb')

In [113]:
fname = 'test.cub'
data = lig_ocup


with open(fname, "w") as cube:
    # first two lines are comments
    cube.write(" Cubefile created by cubetools.py\n  source: none\n")
    natm = len(meta['atoms'])
    nx, ny, nz = data.shape
    cube.write(_putline(natm, *meta['org'])) # 3rd line #atoms and origin
    cube.write(_putline(nx, *meta['xvec']))
    cube.write(_putline(ny, *meta['yvec']))
    cube.write(_putline(nz, *meta['zvec']))
    for atom_mass, atom_pos in meta['atoms']:
        cube.write(_putline(atom_mass, *atom_pos)) #skip the newline
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                if (i or j or k) and k%6==0:
                    cube.write("\n")
                cube.write(" {0: .5E}".format(data[i,j,k]))